## Retirement Calculator

This Python script uses data from Vanguard Target Retirement Funds to calculate estimated retirement income.

In [ ]:
# Libraries
import yfinance as yf
from statistics import mean, stdev
from random import choice
from matplotlib.pyplot import hist, show, xlabel, ylabel, title
from datetime import datetime

# Read in VTTVX Vanguard Target 2025 Fund changes November 1, 2003 - June 30, 2023
vttvxdf = yf.download("VTTVX", end=datetime(2023, 6, 30), interval='1mo')
vttvxdf
vttvxdf['pct_change'] = vttvxdf[['Close']].pct_change()
print(mean(vttvxdf.iloc[1:]['pct_change']))
print(choice(vttvxdf.iloc[1:]['pct_change']))
# hist(vttvxdf.iloc[1:]['pct_change'])
# show()

# Read in Vanguard Target Income Fund Changes July 1, 2007 - June 30, 2023
vtinxdf = yf.download("VTINX", start=datetime(2007, 7, 1), end=datetime(2023, 6, 30), interval='1mo')
vtinxdf
vtinxdf['pct_change'] = vtinxdf[['Close']].pct_change()
print(mean(vtinxdf.iloc[1:]['pct_change']))
print(choice(vtinxdf.iloc[1:]['pct_change']))

# Read in monthly CPI change June 1, 2013 - June 1, 2023 from St. Louis Fed FRED Database
# File stored in Python project directory
inflation=[]
cpichange = open("CPI change.csv", "r")
for line in cpichange:
    rate = line
    rate = float(rate)
    inflation.append(rate)
cpichange.close()
# print(inflation)
print(mean(inflation))
print(stdev(inflation))
print(mean(inflation)*1200) #Inflation rate


In [ ]:
# Functions
def monthcalc(start, end):
    return (end-start)*12

def prerate():
    # return(float(input("Enter rate of return"))) # manual choice of rate
    return(choice(vttvxdf.iloc[1:]['pct_change'])) # use VTTVX monthly rate change data from 2003-2023

def postratefunc():
    # cpi = float(input("Enter inflation"))/12
    # incomerate = float(input("Enter return rate"))
    cpi = choice(inflation)
    incomerate = choice(vtinxdf.iloc[1:]['pct_change'])
    return(incomerate-cpi)

In [ ]:
times = int(input("Enter sim times"))
prebalance = float(input("Enter starting balance"))
contribution = float(input("Enter monthly contribution"))
# startyear = int(input("Current year"))
startyear = datetime.now().year
print("The current year is", startyear)
retyear = int(input("Enter retirement year"))
premonths = monthcalc(startyear,retyear)
print(premonths)
rate = prerate()
ballist =[]
wdlist=[]
for t in range(times):
    balance = prebalance
    for m in range(1,premonths+1):
        rate = prerate()
        # print(rate)
        balance = balance*(1+rate)+contribution
    ballist.append(balance)
    months = monthcalc(retyear + 1, retyear + 35)
    # print(months)
    # withdrawal = balance/months
    # print(withdrawal)
    while months > 0:
        postretrate = postratefunc()
        # print(postretrate)
        withdrawal = balance / months
        wdlist.append(withdrawal)
        balance = (balance*(1+postretrate))-withdrawal
        months -= 1
print(len(ballist))
print(round(mean(ballist), 2))
# print(wdlist)
print(len(wdlist))
print(round(mean(wdlist),2))
hist(wdlist)
title('Retirement Income Simulation')
xlabel('Monthly Withdrawals (Dollars)')
ylabel('Frequency')
show()
